In [1]:
import os.path
from os import path

def make(dirs):
  for dir in dirs:
    if path.exists(dir) == False:
      os.mkdir(dir)

In [44]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

class Model_Simple(nn.Module):
    def __init__(self):
        super(Model_Simple, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.fc1 = nn.LazyLinear(128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output



class Model_CNN(nn.Module):
    def __init__(self):
        super(Model_CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

from collections import defaultdict
class LossCalulcator(nn.Module):
    def __init__(self, temperature, distillation_weight, lossMode="simple", forget=None, teacher=None, device = None):
        super().__init__()
        self.lossMode = lossMode
        self.temperature = temperature
        self.distillation_weight = distillation_weight
        self.loss_log = defaultdict(list)
        self.kldiv = nn.KLDivLoss(reduction='batchmean')
        self.forget = forget
        self.teacher = teacher
        self.device = device

    def simpleDistillLoss(self, outputs, teacher_outputs):
        loss = self.kldiv(F.log_softmax(outputs/self.temperature, dim=1), F.softmax(teacher_outputs/self.temperature, dim=1)) * (self.temperature ** 2)
        return loss

    def influenceFunctionLoss(self, outputs, teacher_outputs):
        if(self.lossMode != "influence" or self.teacher is None or self.forget is None or self.device is None):
            # print("calculating FROM SIMPLE")
            return self.simpleDistillLoss(outputs, teacher_outputs)
        # print("calculating from influence")
        forget_influence = 0
        for batch_idx, (data, target) in enumerate(self.forget):
            data, target = data.to(self.device), target.to(self.device)
            output = self.teacher(data)
            loss = F.nll_loss(output, target)
            forget_influence += loss

        loss = self.kldiv(F.log_softmax(outputs/self.temperature, dim=1), F.softmax(teacher_outputs/self.temperature, dim=1)) * (self.temperature ** 2)
        return loss - forget_influence

    def forward(self, outputs, labels, teacher_outputs=None):
        # Distillation Loss
        soft_target_loss = 0.0
        # print("teacher outputs", teacher_outputs, self.distillation_weight)
        if teacher_outputs is not None and self.distillation_weight > 0.0:
            if(self.lossMode == "simple"):
                soft_target_loss = self.simpleDistillLoss(outputs, teacher_outputs)
            else:
                soft_target_loss = self.influenceFunctionLoss(outputs, teacher_outputs)

        # Ground Truth Loss
        hard_target_loss = F.nll_loss(outputs, labels)

        total_loss = (soft_target_loss * self.distillation_weight) + hard_target_loss

        # # Logging
        if self.distillation_weight > 0:
            self.loss_log['soft-target_loss'].append(soft_target_loss)

        if self.distillation_weight < 1:
            self.loss_log['hard-target_loss'].append(hard_target_loss.item())

        self.loss_log['total_loss'].append(total_loss.item())

        return total_loss

    def get_log(self, length=100):
        log = []
        # calucate the average value from lastest N losses
        for key in self.loss_log.keys():
            if len(self.loss_log[key]) < length:
                length = len(self.loss_log[key])
            log.append("%s: %2.3f"%(key, sum(self.loss_log[key][-length:]) / length))
        return ", ".join(log)


def train(args, model, device, train_loader, optimizer, epoch, teacher=None, lossMode="simple", forget=None, mode="distill"):
    model.train()
    loss_calculator = LossCalulcator(temperature, distillation_weight, lossMode=lossMode, forget=forget, teacher=teacher, device = device).to(device)
    data_loader = train_loader
    if(forget):
        if(mode == "distill"):
            data_loader = train_loader
        elif(mode == "finetune"):
            data_loader =  forget

    for batch_idx, (data, target) in enumerate(data_loader):
        teacher_outputs = None
        # print("train teacher", teacher, distillation_weight)
        if teacher is not None and distillation_weight > 0.0:
            with torch.no_grad():
                # print("getting teacher outputs")
                teacher_outputs = teacher(data.to(device))
                # print("teacher outputs", teacher_outputs)

        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        # loss = F.nll_loss(output, target)
        loss = loss_calculator(outputs          = output,
                               labels           = target.to(device),
                               teacher_outputs  = teacher_outputs)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}, LossLogs: {}'.format(
                epoch, batch_idx * len(data), len(data_loader.dataset),
                100. * batch_idx / len(data_loader), loss.item(), loss_calculator.get_log()))
            if args.dry_run:
                break


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

class DistArgs:
    batch_size = 64
    test_batch_size = 1000
    epochs = 5
    lr = 1.0
    gamma = 0.7
    no_cuda = False
    no_mps = False
    dry_run = False
    # seed = 1
    log_interval = 10
    save_model = False


def Distill(saveFile, teacher=None, mode="distill",lossMode="simple", forgetDataCount=None, trainData=None, testData=None):
    # Training settings
    args = DistArgs()
    use_cuda = not args.no_cuda and torch.cuda.is_available()
    use_mps = not args.no_mps and torch.backends.mps.is_available()

    # torch.manual_seed(args.seed)

    if use_cuda:
        device = torch.device("cuda")
    elif use_mps:
        device = torch.device("mps")
    else:
        device = torch.device("cpu")

    train_kwargs = {'batch_size': args.batch_size}
    forget_kwargs = {'batch_size': args.batch_size}
    test_kwargs = {'batch_size': args.test_batch_size}
    if use_cuda:
        cuda_kwargs = {'num_workers': 1,
                       'pin_memory': True,
                       'shuffle': True}
        train_kwargs.update(cuda_kwargs)
        test_kwargs.update(cuda_kwargs)

    dataset1 = trainData
    dataset2 = testData
    train_loader =  torch.utils.data.DataLoader(dataset1,**train_kwargs) if trainData else None
    test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

    forget_loader = None
    if(forgetDataCount is not None):
        split = torch.utils.data.random_split(dataset1, [forgetDataCount/len(dataset1), 1 - forgetDataCount/len(dataset1)])
        split = split[0]
        forget_loader = torch.utils.data.DataLoader(split, **forget_kwargs)
    model = Model_CNN().to(device)
    if(teacher is not None):
        print("Forgetting process started")
        if(mode == "distill"):
            model = Model_Simple().to(device)
        elif(mode == "finetune"):
            # intialise student with teacher weights
            student = teacher
            model = student.to(device)

    optimizer = optim.Adadelta(model.parameters(), lr=args.lr)

    scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)
    for epoch in range(1, args.epochs + 1):
        train(args, model, device, train_loader, optimizer, epoch, teacher=teacher,  lossMode=lossMode, forget=forget_loader, mode=mode)
        test(model, device, test_loader)
        scheduler.step()

    if args.save_model:
        torch.save(model.state_dict(), saveFile + ".pt")
    return model


In [39]:
#  Teacher model

temperature = 0
distillation_weight = 0
transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
dataset1 = datasets.MNIST('../data', train=True, download=True,
                       transform=transform)
dataset2 = datasets.MNIST('../data', train=False,
                       transform=transform)
teacher = Distill("mnist_cnn", teacher=None, lossMode="simple", forgetDataCount=None, trainData=dataset1, testData=dataset2)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.301005, LossLogs: hard-target_loss: 2.301, total_loss: 2.301
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.414332, LossLogs: hard-target_loss: 2.030, total_loss: 2.030
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.945560, LossLogs: hard-target_loss: 1.581, total_loss: 1.581
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.545673, LossLogs: hard-target_loss: 1.295, total_loss: 1.295
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.631308, LossLogs: hard-target_loss: 1.106, total_loss: 1.106
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.407871, LossLogs: hard-target_loss: 0.973, total_loss: 0.973
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.377798, LossLogs: hard-target_loss: 0.878, total_loss: 0.878
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.360814, LossLogs: hard-target_loss: 0.795, total_loss: 0.795
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.452560, LossLogs: hard-target_loss: 0.741, total_loss: 0.741
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.246740, LossLogs: hard-target_loss: 0.698, t

In [45]:
#  Student Distillation Loss via simple NLL
# Test set: Average loss: 0.0393, Accuracy: 9890/10000 (99%)

distillation_weight = 0.1
temperature = 1
transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
dataset1 = datasets.MNIST('../data', train=True, download=True,
                       transform=transform)
dataset2 = datasets.MNIST('../data', train=False,
                       transform=transform)

studentSimpleLoss = Distill("studentSimple_mnist_cnn", teacher=teacher, mode="distill", lossMode="simple", forgetDataCount=150, trainData=dataset1, testData=dataset2)


Forgetting process started
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.575476, LossLogs: soft-target_loss: 2.328, hard-target_loss: 2.343, total_loss: 2.575
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.440665, LossLogs: soft-target_loss: 2.400, hard-target_loss: 2.422, total_loss: 2.662
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.928657, LossLogs: soft-target_loss: 1.755, hard-target_loss: 1.772, total_loss: 1.947
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.601277, LossLogs: soft-target_loss: 1.411, hard-target_loss: 1.431, total_loss: 1.573
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.544843, LossLogs: soft-target_loss: 1.212, hard-target_loss: 1.233, total_loss: 1.354
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.525859, LossLogs: soft-target_loss: 1.074, hard-target_loss: 1.097, total_loss: 1.204
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.394726, LossLogs: soft-target_loss: 0.986, hard-target_loss: 1.010, total_loss: 1.108
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.353995, LossLogs: soft-target_loss: 0.911, hard

In [46]:
#  Student Distillation Loss via error maximising loss for forget dataset
# Test set: Average loss: 0.0416, Accuracy: 9861/10000 (99%)


distillation_weight = 0.1
temperature = 1
transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
dataset1 = datasets.MNIST('../data', train=True, download=True,
                       transform=transform)
dataset2 = datasets.MNIST('../data', train=False,
                       transform=transform)

studentInfluenceLoss = Distill("studentInfluence_mnist_cnn", teacher=teacher, mode="distill", lossMode="influence", forgetDataCount=150, trainData=dataset1, testData=dataset2)


Forgetting process started
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.570347, LossLogs: soft-target_loss: 2.268, hard-target_loss: 2.344, total_loss: 2.570
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.784763, LossLogs: soft-target_loss: 2.351, hard-target_loss: 2.434, total_loss: 2.669
Train Epoch: 1 [1280/60000 (2%)]	Loss: 1.107945, LossLogs: soft-target_loss: 1.756, hard-target_loss: 1.840, total_loss: 2.015
Train Epoch: 1 [1920/60000 (3%)]	Loss: 1.116947, LossLogs: soft-target_loss: 1.468, hard-target_loss: 1.550, total_loss: 1.696
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.444855, LossLogs: soft-target_loss: 1.235, hard-target_loss: 1.315, total_loss: 1.439
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.393068, LossLogs: soft-target_loss: 1.068, hard-target_loss: 1.151, total_loss: 1.257
Train Epoch: 1 [3840/60000 (6%)]	Loss: 1.210882, LossLogs: soft-target_loss: 0.982, hard-target_loss: 1.066, total_loss: 1.164
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.274041, LossLogs: soft-target_loss: 0.892, hard

In [48]:
#  Student Distillation Loss via error maximising loss for large forget dataset
# Test set: Average loss: 0.0416, Accuracy: 9861/10000 (99%)

distillation_weight = 0.1
temperature = 1
transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
dataset1 = datasets.MNIST('../data', train=True, download=True,
                       transform=transform)
dataset2 = datasets.MNIST('../data', train=False,
                       transform=transform)

studentInfluenceLoss_large = Distill("studentInfluence_large_mnist_cnn", teacher=teacher, mode="distill", lossMode="influence", forgetDataCount=1000, trainData=dataset1, testData=dataset2)


Forgetting process started


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.499763, LossLogs: soft-target_loss: 1.727, hard-target_loss: 2.327, total_loss: 2.500
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.460172, LossLogs: soft-target_loss: 1.536, hard-target_loss: 2.121, total_loss: 2.275
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.792871, LossLogs: soft-target_loss: 1.119, hard-target_loss: 1.703, total_loss: 1.815
Train Epoch: 1 [1920/60000 (3%)]	Loss: 1.168129, LossLogs: soft-target_loss: 0.804, hard-target_loss: 1.388, total_loss: 1.468
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.303770, LossLogs: soft-target_loss: 0.621, hard-target_loss: 1.202, total_loss: 1.265
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.458391, LossLogs: soft-target_loss: 0.478, hard-target_loss: 1.059, total_loss: 1.107
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.477886, LossLogs: soft-target_loss: 0.369, hard-target_loss: 0.948, total_loss: 0.985
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.485035, LossLogs: soft-target_loss: 0.294, hard-target_loss: 0.874, total_